In [3]:
from pyspark.sql import SparkSession
import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn
from mlflow.models.model import get_model_info
from mlflow.models import infer_signature, set_signature

from databricks.connect import DatabricksSession

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
df = pd.read_csv("brasil-real-estate-1.csv")

In [ ]:
df['price_usd'] = (df['price_usd']
                    .str.replace("$", "", regex = False)
                    .str.replace(",", "")
                    .astype(float)
)

In [ ]:
df.drop(columns=["place_with_parent_names","region","lat-lon"],inplace = True)
df.head()

y = df['price_usd']
X = df.drop(columns=["price_usd"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Set the experiment name dynamically based on the model name
experiment_name = "/Workspace/Users/abdulafeez.adefabi@pwc.com/Personal/trial"
mlflow.set_experiment(experiment_name)

# Start MLflow run
with mlflow.start_run(run_name="RidgeModel"):
    # Create and fit the model
    alpha_value = 1.0  # Example value, replace with the actual alpha used
    model = make_pipeline(OneHotEncoder(handle_unknown='ignore'), SimpleImputer(), Ridge(alpha=alpha_value))
    model.fit(X_train, y_train)

    # Log the model with a signature
    mlflow.sklearn.log_model(model, "model", signature=infer_signature(X_train, model.predict(X_train)))

    # Log hyperparameters
    mlflow.log_param("alpha", alpha_value)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    mse_value = mean_squared_error(y_test, y_pred)
    r2_value = r2_score(y_test, y_pred)
    rmse_value = mean_squared_error(y_test, y_pred, squared=False)
    mae_value = mean_absolute_error(y_test, y_pred)

    mlflow.log_metric("mse", mse_value)
    mlflow.log_metric("r2", r2_value)
    mlflow.log_metric("rmse", rmse_value)
    mlflow.log_metric("mae", mae_value)

    # Get the run ID and construct the model URI
    run_id = mlflow.active_run().info.run_id
    model_uri = f"runs:/{run_id}/model"
    print(model_uri)

    # Register the model
    registered_model = mlflow.register_model(model_uri, "Predicting_House")
    
    # Serve the model
    mlflow.pyfunc.serve(model_uri=model_uri, host="0.0.0.0", port=5000)